In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
device = "cpu"

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [4]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [27]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
# opt = torch_numopt.ConjugateGradient(model.parameters(), lr=1e-4, model=model, c1=1e-4, tau=0.1, line_search_method="const", cg_method="PR")
opt = torch_numopt.ConjugateGradient(model.parameters(), lr=10, model=model, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="armijo", cg_method="PR")
# opt = torch_numopt.ConjugateGradient(model.parameters(), lr=10, model=model, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="wolfe", cg_method="PR")
# opt = torch_numopt.ConjugateGradient(model.parameters(), lr=10, model=model, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="strong-wolfe", cg_method="PR")
# opt = torch_numopt.ConjugateGradient(model.parameters(), lr=10, model=model, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="goldstein", cg_method="PR")


all_loss = {}
for epoch in range(1000):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].detach().numpy().item()))

epoch:  0, loss: 0.09446775168180466
epoch:  1, loss: 0.06652283668518066
epoch:  2, loss: 0.05295877903699875
epoch:  3, loss: 0.04589714854955673
epoch:  4, loss: 0.04201127961277962
epoch:  5, loss: 0.039848800748586655
epoch:  6, loss: 0.03863472118973732
epoch:  7, loss: 0.03794476389884949
epoch:  8, loss: 0.03754350543022156
epoch:  9, loss: 0.037298452109098434
epoch:  10, loss: 0.03713229298591614
epoch:  11, loss: 0.03688928112387657
epoch:  12, loss: 0.03675800561904907
epoch:  13, loss: 0.0366847887635231
epoch:  14, loss: 0.03664105758070946
epoch:  15, loss: 0.036586277186870575
epoch:  16, loss: 0.03651970252394676
epoch:  17, loss: 0.03647948428988457
epoch:  18, loss: 0.036419447511434555
epoch:  19, loss: 0.036356303840875626
epoch:  20, loss: 0.03631788119673729
epoch:  21, loss: 0.03624971956014633
epoch:  22, loss: 0.036192089319229126
epoch:  23, loss: 0.03617352619767189
epoch:  24, loss: 0.03607888147234917
epoch:  25, loss: 0.03602500259876251
epoch:  26, loss:

In [28]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.8317877499343602
Test metrics:  R2 = 0.8622204538237305


In [25]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
# opt = torch_numopt.ConjugateGradient(model.parameters(), lr=1e-4, model=model, c1=1e-4, tau=0.1, line_search_method="const", cg_method="FR")
opt = torch_numopt.ConjugateGradient(model.parameters(), lr=10, model=model, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="armijo", cg_method="FR")
# opt = torch_numopt.ConjugateGradient(model.parameters(), lr=10, model=model, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="wolfe", cg_method="FR")
# opt = torch_numopt.ConjugateGradient(model.parameters(), lr=10, model=model, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="strong-wolfe", cg_method="FR")
# opt = torch_numopt.ConjugateGradient(model.parameters(), lr=10, model=model, c1=1e-4, tau=0.1, line_search_method="backtrack", line_search_cond="goldstein", cg_method="FR")


all_loss = {}
for epoch in range(1000):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].detach().numpy().item()))

epoch:  0, loss: 0.3300580680370331
epoch:  1, loss: 0.1837853640317917
epoch:  2, loss: 0.11212076246738434
epoch:  3, loss: 0.07540986686944962
epoch:  4, loss: 0.05648688226938248
epoch:  5, loss: 0.046762801706790924
epoch:  6, loss: 0.041798390448093414
epoch:  7, loss: 0.039280347526073456
epoch:  8, loss: 0.03800981491804123
epoch:  9, loss: 0.03737065941095352
epoch:  10, loss: 0.03704943507909775
epoch:  11, loss: 0.036887358874082565
epoch:  12, loss: 0.03680473193526268
epoch:  13, loss: 0.03676164522767067
epoch:  14, loss: 0.03673812374472618
epoch:  15, loss: 0.0367242731153965
epoch:  16, loss: 0.03670588880777359
epoch:  17, loss: 0.03668253496289253
epoch:  18, loss: 0.03666846826672554
epoch:  19, loss: 0.03664163500070572
epoch:  20, loss: 0.03661837428808212
epoch:  21, loss: 0.03660411387681961
epoch:  22, loss: 0.036563947796821594
epoch:  23, loss: 0.03654107451438904
epoch:  24, loss: 0.03652636706829071
epoch:  25, loss: 0.03648769482970238
epoch:  26, loss: 0.

In [26]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.8359038322267865
Test metrics:  R2 = 0.8463810076509527
